In [396]:
from Crypto.Util import number
from Crypto import Random
from random import random
#from Crypto.Random import random 
from Crypto.PublicKey import ElGamal
from Crypto.Util.number import GCD
# from Crypto.Hash import SHA

#### ElGamal key generator with precalculated p

Next part is taken from the package itself. The aim of this modification is to allow to generate ElGamal keys with current p.
We use current p as big prime value search is too time-consuming during testing, while it can be easily switched back after testing.

In [397]:
p = 141176738696375781725204243127769890268678619091990255801090867725791766860711540511936752690462516778314493729915532970792016127308909781866550288155536629472896705613223748593785835919511841224246096551394931332973439359387621004641147628168972254601172019075209676946914948899011670138359243952372297804367

from Crypto.PublicKey.ElGamal import number
from Crypto.PublicKey.ElGamal import ElGamalobj

def GenerateKey(p = None):
    if p == None:
        return ElGamal.generate(1024, Random.new().read)
    else:
        key = ElGamalobj()
        key.p = p
        key.x = number.getRandomRange(2, p-1, Random.new().read)
        while 1:
            key.g = number.getRandomRange(3, key.p, Random.new().read)
            safe = 1
            if pow(key.g, 2, key.p)==1:
                safe=0
            if safe and pow(key.g, (p-1) // 2, key.p)==1:
                safe=0
            if safe and divmod(key.p-1, key.g)[1]==0:
                safe=0
            ginv = number.inverse(key.g, key.p)
            if safe and divmod(key.p-1, ginv)[1]==0:
                safe=0
            if safe:
                break
        key.y = pow(key.g, key.x, key.p)
        return key

#### Bets

We are still thinking about creating a special class for bets.
But surely there are next functions, related to it.

Structure of a bet:

For further implementations with alive or at least virtual players bet should be text. 
That is easy to change, but now for convenience a bet is a dictionary of these values:
'name', 'event', 'outcome', 'size of bet', 'public key', 'signature'. 

In [408]:
def Hash_no_sig_bet(bet):
    text = str(bet['event']) + ' ' + str(bet['outcome']) + \
    ' ' + str(bet['size of bet']) + \
    ' ' + str(bet['name']) + ' ' + str(bet['public key']) # this line seems to be excess since changing it is useless
    return hashlib.sha256(text.encode('utf-8')).digest()

# requires full key
def Sign_bet(key, bet):
    h = Hash_no_sig_bet(bet)
    while 1:
        k = Random.random.StrongRandom().randint(1, key.p-1)
        if GCD(k, key.p-1) == 1: break
    return key.sign(h, k)

#requires only public part of the key
def Verify_bet(key, bet):
    h = Hash_no_sig_bet(bet)
    return key.verify(h, bet['signature'])

def Hash_bet(bet):
    text = str(bet['event']) + ' ' + str(bet['outcome']) + ' ' + str(bet['size of bet']) + ' ' + str(bet['signature']) + \
    ' ' + str(bet['name']) + ' ' + str(bet['public key']) # this line seems to be excess since changing it is useless
    return hashlib.sha256(text.encode('utf-8')).hexdigest()

#### Events

There are also event subblock in the block. 

In [399]:
def Hash_event(event, outcome, bet_pointers):
    text = str(event) + ' ' + str(outcome) + ' ' + str(bet_pointers)
    return hashlib.sha256(text.encode('utf-8')).hexdigest()

#### Block

Block consists of next parts: 
miner name,
hash,
bets,
previous block hash,
target,
closing events.

In [517]:
hash_prefix_limit = 10**20

class Block:
    def __init__(self, miner_name, player_num, previous_block_hash, target):
        self.miner_name = miner_name
        self.player_num = player_num
        self.previous_block_hash = previous_block_hash
        self.target = target
        
        self.bets = []
        self.bet_hash = hashlib.sha256(''.encode('utf-8'))
        self.closing_events = {}
        self.events_hash_order = []
        self.event_hash = hashlib.sha256(''.encode('utf-8'))
        self.Hash = hashlib.sha256(''.encode('utf-8'))
        self.prefix = ''
    
    def add_bet(self, bet):
        self.bets.append(bet)
        #print('str(Hash_bet(bet))', str(Hash_bet(bet)))
        self.bet_hash = hashlib.sha256((self.bet_hash.hexdigest() + str(Hash_bet(bet))).encode('utf-8'))
        true_hash = self.GetRecalculatedBetHash()
        #print(true_hash.hexdigest(), self.bet_hash.hexdigest())
        #if true_hash.hexdigest() == self.bet_hash.hexdigest():
        #    print('Succesful add bet')
        #else:
        #    print('Unsuccesful add bet')
    
    def GetRecalculatedBetHash(self):
        bet_hash = hashlib.sha256(''.encode('utf-8'))
        for bet in self.bets:
            #print('str(Hash_bet(bet))', str(Hash_bet(bet)))
            bet_hash = hashlib.sha256((bet_hash.hexdigest() + str(Hash_bet(bet))).encode('utf-8'))
        return bet_hash
    
    def GetRecalculatedEventHash(self):
        event_hash = hashlib.sha256(''.encode('utf-8'))
        for event in self.events_hash_order:
            outcome, bet_pointers = self.closing_events[event]
            event_hash = hashlib.sha256((event_hash.hexdigest() + \
                                        str(Hash_event(event, outcome, bet_pointers))).encode('utf-8'))
        return event_hash

    def HashwithPrefix(self, prefix):
        text = str(prefix) + ' ' + str(self.player_num) + ' ' +  str(self.bet_hash) +\
        ' ' + str(self.previous_block_hash) + ' ' + ' ' + str(self.event_hash) + ' ' + str(self.target)
        return hashlib.sha256(text.encode('utf-8'))
    
    def Verify(self, target):
        true_bet_hash = self.GetRecalculatedBetHash()
        if true_bet_hash.hexdigest() != self.bet_hash.hexdigest():
            print('hash bet verify failed')
            return False
        
        true_event_hash = self.GetRecalculatedEventHash()
        if true_event_hash.hexdigest() != self.event_hash.hexdigest():
            print('hash event verify failed')
            return False

        true_hash = self.HashwithPrefix(self.prefix).hexdigest()
        #print('true_hash', true_hash)
        #print('self.Hash.hexdigest()', self.Hash.hexdigest())
        #print('int(self.Hash, 16)', int(self.Hash.hexdigest(), 16))
        #print('self.target', self.target)
        #print('target')
        return self.Hash.hexdigest() == true_hash and int(self.Hash.hexdigest(), 16) < self.target and self.target == target

    def TryToSign(self, target):
        prefix = str(randint(0, hash_prefix_limit))
        h = self.HashwithPrefix(prefix)
        if int(h.hexdigest(), 16) < target: 
            self.prefix = prefix
            self.Hash = h
            return True
        return False
    
    def CloseEvent(self, event, outcome, tokenprize, bet_pointers):
        print('CloseEvent')
        self.closing_events[event] = ((outcome, bet_pointers))
        self.event_hash = hashlib.sha256((self.event_hash.hexdigest() + \
                                         str(Hash_event(event, outcome, bet_pointers))).encode('utf-8'))
        self.delete_outdated_bets(event)
        self.events_hash_order.append(event)
        return len(self.closing_events)
    
    #This method support use of Merkle tree
    #I can't just delete them because it is slow and the pointers to the bets will break
    #The last problem is easily fixed but I pray this project at least works when deadline comes
    def delete_outdated_bets(self, event):
        for bet_num in range(len(self.bets)):
            if self.bets[bet_num]['event'] == event:
                self.bets[bet_num]['size of bet'] = 0
        self.bet_hash = self.GetRecalculatedBetHash()

### Emulators
Classes and databases to emulate actual players and miners.

#### Class Player.

While were are not ready to implement fully virtual players or interface for real players, we create puppets.
Each player will bet his money (balance) on some outcome of events.
He has a private and a corresponding public key to sign his bets. 

There is a possible variation in implementation: bets on negative outcome may be considered as negative size of bet on positive outcome. But for readbility this option was denied.

In [451]:
class Player:
    def __init__(self, name, p, balance, bet_probability):
        self.name = name
        key = GenerateKey(p)
        self.key = key
        self.private_key = key.x
        self.public_key = key.p, key.g, key.y
        self.balance = balance
        self.bets = {}
        self.bet_probability = bet_probability
    
    # opening_events is a list, closing_events is a dictionary
    def update(self, closing_events_token_price):
        for closing_event_number in closing_events_token_price:
            token_prize = closing_events_token_price[closing_event_number]
            if closing_event_number in self.bets:
                self.balance += self.bets.pop(closing_event_number) * token_prize
    
    def bet(self, opening_events):
        bets = []
        for event_number in opening_events:
            size_of_bet = 10
            if random() < self.bet_probability and self.balance >= size_of_bet:
                outcome = 2 * randint(0, 1) - 1
                bet = {'name':self.name, 'event':event_number, 'outcome':outcome,
                       'size of bet':size_of_bet, 'public key':self.public_key}
                bet['signature'] = Sign_bet(self.key, bet)
                bets.append(bet)
                if event_number not in self.bets:
                    self.bets[event_number] = 0
                self.bets[event_number] += size_of_bet * outcome
                self.balance -= size_of_bet
        return bets
    
# надо написать чтобы игрок делал ставки не только на только что появившиеся евенты
# передавать ему не открывающиеся, а открытые мб?

#### Miners opinion on players balance, public key

In [402]:
class PlayersData:
    def __init__(self, initial_balance):
        self.initial_balance = initial_balance
        self.balances = {}
        
    def UpdateAfterEvent(self, outcome, bet):
        self.balances[bet['name']][0] += bet['size of bet'] * outcome * bet['outcome']
    
    def CanBet(self, bet):
        if not bet['name'] in self.balances:
            self.balances[bet['name']] = [self.initial_balance, bet['public key']]
        return self.balances[bet['name']][0] >= bet['size of bet']
    
    def Bet(self, bet):
        if not bet['name'] in self.balances:
            self.balances[bet['name']] = [self.initial_balance, bet['public key']]
        self.balances[bet['name']][0] -= bet['size of bet']
    
    def VerifyPublicKey(self, bet):
        name = bet['name']
        if bet['name'] in self.balances:
            if bet['public key'] != self.balances[name][1]:
                return False
        return True

майнер хранит ссылки на все закрытия, чтобы проверять какие ставки не валидны

сейчас нет проверки на неповторяемость

bet_pointer = ((block_num, bet_num))

может нужен отдельный класс для евентов. сейчас это 4 позиции: 
статус(открыт-закрыт), 
массив указателей на ставки с этим событием,
цена токена,
позиция закрытия евента в блокчейне

#### Miner

In [515]:
class Miner:
    def __init__(self, name, player_num, initial_balance, n_events, GenesisBlock, target, neighbors):
        self.name = name
        self.initial_balance = initial_balance
        self.events = [['open', [], None, None] for i in range(n_events)]
        self.target = target
        self.player_num = player_num
        self.neighbors = neighbors
        
        self.blockchain = [GenesisBlock] # Genesis Block. Not sure if it is the right place for him
        self.player_data = PlayersData(self.initial_balance)
        self.block = Block(self.name, player_num, GenesisBlock.Hash, target)
        self.balances = PlayersData(initial_balance)

    def CloseEvents(self, closing_events):
        for event in closing_events:
            outcome = closing_events[event]
            bet_pointers = self.events[event][1]
            if self.events[event][0] == 'open':
                self.events[event] = ['closed', bet_pointers, outcome, 0]
                token_prize = self.GetTokenPrize(event)
                event_number = self.block.CloseEvent(event, outcome, token_prize, bet_pointers)
                self.events[event][3] = ((len(self.blockchain), event_number))
                self.block.CloseEvent(event, outcome, token_prize, bet_pointers)
                

    def GetTokenPrize(self, event):
        betsbyoutcome = [0, 0]
        true_block_pointers = []
        for bet_pointer in self.events[event][1]:
            block_number, bet_number = bet_pointer
            if block_number < len(self.blockchain):
                true_block_pointers.append(bet_pointer)
                bet = self.blockchain[block_number].bets[bet_number]
                betsbyoutcome[bet['outcome']] += bet['size of bet']
        outcome = self.events[event][2]
        totaly_bet = betsbyoutcome[0] + betsbyoutcome[1]
        self.events[event][1] = true_block_pointers
        if totaly_bet < 1:
            return 0
        if betsbyoutcome[outcome] == 0:
            return 0
        return (totaly_bet - 1) / (betsbyoutcome[outcome])
    # -1 because miner takes money
    
    def EventUpdateWallets(self, outcome, bet_pointers):
        for bet_pointer in bet_pointers:
            block_number, bet_number = bet_pointer
            bet = self.blockchain[block_number].bets[bet_number]
            self.player_data.UpdateAfterEvent(outcome, bet)
    
    def add_bet(self, bet):
        if not self.verify_bet(bet):
            print('Unexpected Bad Bet')
            return False
        # add_event(bet['event']) must be here if events become dynamical
        self.events[bet['event']][1].append((len(self.blockchain), len(self.block.bets)))
        self.block.add_bet(bet)
        self.player_data.Bet(bet)
        return True

    def verify_bet(self, bet):
        if not self.player_data.VerifyPublicKey(bet):
            return False
        key = ElGamalobj()
        key.p, key.g, key.y = bet['public key']
        if not Verify_bet(key, bet):
            return False
        if bet['size of bet'] < 0:
            return False
        if self.events[int(bet['event'])][0] != 'open':
            return False
        if not self.player_data.CanBet(bet):
            return False
        return True
    
    def AddBlock(self):
        self.blockchain.append(self.block)
        self.block = Block(self.name, self.player_num, 
                           self.blockchain[-1].Hash, self.target)
        
    def Mine(self):
        success = self.block.TryToSign(self.target)
        if success:
            print('Successful mining')
            self.AddBlock()
            self.SendBlock()
        return success
    
    def RecalculateBalances(self):
        self.player_data = PlayersData(self.initial_balance)
        for block in self.blockchain:
            for bet in block.bets:
                self.player_data.Bet(bet)
            for event in block.closing_events:
                outcome, bets = block.closing_events[event]
                self.player_data.UpdateAfterEvent(outcome, bets)
    
    # Upon sending a pointer to himself is sent to let the neighbor contact back
    def SendBlock(self):
        print('SendBlock', len(self.neighbors))
        for miner in self.neighbors:
            miner.ReceiveBlock(self, self.blockchain[-1])

    def ReceiveBlock(self, neighbor, block):
        if block.previous_block_hash != self.blockchain[-1].Hash and \
           block.Hash != self.blockchain[-1].Hash:
            # Unexpected for this model Disconsensus!
            # should be implemented in normal blockchain
            print('Strong Disconsensus')
            return
        # Should also check wether all bets are possible. 
        # But in our model there are no bad miners still.
        if block.previous_block_hash == self.blockchain[-1].Hash:
            #print('Verifing...')
            if block.Verify(self.target):
                print('Acceptance')
                self.block = block
                self.AddBlock()
                self.RecalculateBalances()
                self.SendBlock()
                return
            print('Veryfing failed')

#### EventGenerator

Generates numbers of events, which become open or trigger round.
Any event have one of three possible statuses: not open, open, closed.
Every event starts as not open before the first round.
Some round it becomes open, and aligble for betting on.
And some othey round it becomes closed. His outcomes becomes known.
And it is now forbidden to bet on him.
Each time yield() is called the generator returns two objects:

1) A list of events opening this round

2) A dictionary, whose keys are numbers of events which become close this round, 
and values are the outcomes of corresponding events. For convenience outcome value is either +1 or -1.

In [404]:
from random import randint

def EventGenerator(n_events, n_rounds):
    times = [sorted((randint(0, n_rounds - 2), randint(1, n_rounds - 2))) for i in range(n_events)]
    for i in range(n_events):
        times[i][1] += 1
    calls = [[[],[]] for i in range(n_rounds)]
    for event in range(n_events):
        calls[times[event][0]][0].append(event)
        calls[times[event][1]][1].append(event)
    for round_number in range(n_rounds):
        yield(calls[round_number][0], {i:(2 * randint(0,1) - 1) for i in calls[round_number][1]})

#### Consensus network mechanism

### Main environment itself

In [518]:
def Main(n_miners = 5, miners_network_topology=[[(i - 1) % 5, (i + 1) % 5] for i in range(5)], 
         n_players = 10, n_events = 10, n_rounds = 10, initial_balance = 1000, bet_probability = 1.,
         target = 2**255, n_attemps = 1): 
    GenesisBlock = Block('', -1, '', 0)
    GenesisBlock.TryToSign(2**256)
    miners = [Miner('Miner' + str(i), i, initial_balance, n_events, GenesisBlock, target, []) for i in range(n_miners)]
    # Dangerous moment, checked this, but still afraid that pointers don't work like this in python.
    for miner_num in range(n_miners):
        for miner_neighbor_num in miners_network_topology[miner_num]:
            miners[miner_num].neighbors.append(miners[miner_neighbor_num])
    players = [Player('Player' + str(i), p, initial_balance, bet_probability) for i in range(n_players)]
    Bet_pool = []
    event_generator = EventGenerator(n_events, n_rounds)
    for round_number in range(n_rounds):
        # Events initiation
        opening_events, closing_events = next(event_generator)
        
        # Miners close events
        for miner_num in range(n_miners):
            miners[miner_num].CloseEvents(closing_events)
            
        # Players open, close events
        closing_events_token_price = {event:miners[0].GetTokenPrize(event) for event in closing_events}
        for player_num in range(n_players):
            players[player_num].update(closing_events_token_price)
            
        # Players Bet
        Bet_pool.append([])
        for player_num in range(n_players):
            new_bets = players[player_num].bet(opening_events)
            Bet_pool[-1] = Bet_pool[-1] + new_bets
        
        # Miners read Bets
        for miner_num in range(n_miners):
            for bet in Bet_pool[-1]:
                miners[miner_num].add_bet(bet)
        # Miners mine
        #if at least one mining was succesful then the consensus is lost and
        #for purpose of this project it must immediately be restored
        succesful_mining = False
        for miner_num in range(n_miners):
            for attempt in range(n_attemps):
                for miner_num_i in range(n_miners):
                    print(len(miners[miner_num_i].blockchain),end = ' ')
                print('')
                if miners[miner_num].Mine():
                    succesful_mining = True
    return miners, Bet_pool
miners, BP = Main()

1 1 1 1 1 
Successful mining
SendBlock 2
Acceptance
SendBlock 2
Acceptance
SendBlock 2
Acceptance
SendBlock 2
Acceptance
SendBlock 2
2 2 2 2 2 
Successful mining
SendBlock 2
Acceptance
SendBlock 2
Acceptance
SendBlock 2
Acceptance
SendBlock 2
Acceptance
SendBlock 2
3 3 3 3 3 
Successful mining
SendBlock 2
Acceptance
SendBlock 2
Acceptance
SendBlock 2
Acceptance
SendBlock 2
Acceptance
SendBlock 2
4 4 4 4 4 
4 4 4 4 4 
4 4 4 4 4 
4 4 4 4 4 
4 4 4 4 4 
4 4 4 4 4 
4 4 4 4 4 
Successful mining
SendBlock 2
Acceptance
SendBlock 2
Acceptance
SendBlock 2
Acceptance
SendBlock 2
Acceptance
SendBlock 2
5 5 5 5 5 
Successful mining
SendBlock 2
Acceptance
SendBlock 2
Acceptance
SendBlock 2
Acceptance
SendBlock 2
Acceptance
SendBlock 2
6 6 6 6 6 
6 6 6 6 6 
Successful mining
SendBlock 2
Acceptance
SendBlock 2
Acceptance
SendBlock 2
Acceptance
SendBlock 2
Acceptance
SendBlock 2
7 7 7 7 7 
7 7 7 7 7 
Successful mining
SendBlock 2
Acceptance
SendBlock 2
Acceptance
SendBlock 2
Acceptance
SendBlock 2
Acce

TypeError: list indices must be integers or slices, not str

In [504]:
for i in range(len(miners)):
    print(len(miners[i].blockchain))

5
7
3
7
6
